# **LP4_Run3**

In [10]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))



### **Generate Mathematical Model**

In [ ]:
problem = """ Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets.

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs)."""

In [47]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [48]:
print(response1.choices[0].message.content)

Mathematical Optimization Model:

Parameters:
- Yields (in T/acre): y_w = 2.5 for wheat, y_c = 3 for corn, y_s = 20 for sugar beets
- Net Selling Prices (in $/T): p_w = $170 for wheat, p_c = $150 for corn, p_s = $36 for sugar beets (quota price), and p_sq = $10 for sugar beets (excess quota price)
- Purchase Prices (in $/T): pw_w = $170*1.4 for wheat, pw_c = $150*1.4 for corn
- Planting Costs (in $/acre): c_w = $150 for wheat, c_c = $230 for corn, c_s = $260 for sugar beets
- Total available land (in acres): A = 500
- Quota for sugar beets (in T): Q_s = 6000
- Demand for wheat and corn (in T): D_w = 200, D_c = 240

Decision Variables:
- Land allocated for each crop (in acres): L_w, L_c, L_s
- Quantity of each crop to be produced (P_w, P_c, P_s) and purchased (pw_w, pw_c)

Objective (Minimize): 
- Total Costs = L_w*c_w + L_c*c_c + L_s*c_s + pw_w*p_w +pw_c*p_c - p_w*P_w – p_c*P_c – p_s*min(Q_s, P_s) – p_sq*max(0, P_s– Q_s)

Constraints:
- Land constraint: L_w + L_c + L_s ≤ A
- Production

### **Generate Pyomo Code**

In [49]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [51]:
print(response2.choices[0].message.content)

Python Pyomo Code:

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define parameters (using sample data)
c_w = 150
c_c = 230
c_s = 260
A = 500
y_w = 2.5
y_c = 3
y_s = 20
p_w = 170
p_c = 150
p_s = 36
p_sq = 10
pw_w = p_w * 1.4
pw_c = p_c * 1.4
Q_s = 6000
D_w = 200
D_c = 240

# Define variables
model.L_w = Var(domain=NonNegativeReals)
model.L_c = Var(domain=NonNegativeReals)
model.L_s = Var(domain=NonNegativeReals)
model.pw_w = Var(domain=NonNegativeReals)
model.pw_c = Var(domain=NonNegativeReals)

# Define constraints 
model.land_con = Constraint(expr = model.L_w + model.L_c + model.L_s <= A)
model.prod_amount_wheat = Constraint(expr = model.L_w*y_w >= D_w - model.pw_w)
model.prod_amount_corn = Constraint(expr = model.L_c*y_c >= D_c - model.pw_c)
model.prod_amount_sugar_beets = Constraint(expr = model.L_s*y_s >= Q_s)
model.demand_req_wheat = Constraint(expr = y_w*model.L_w + model.pw_w >= D_w)
model.demand_req_corn = Constraint(expr = y_c*model

### **Run the code Generated by GPT4**

In [53]:
from pyomo.environ import *
model = ConcreteModel()

# Define parameters (using sample data)
c_w = 150
c_c = 230
c_s = 260
A = 500
y_w = 2.5
y_c = 3
y_s = 20
p_w = 170
p_c = 150
p_s = 36
p_sq = 10
pw_w = p_w * 1.4
pw_c = p_c * 1.4
Q_s = 6000
D_w = 200
D_c = 240

# Define variables
model.L_w = Var(domain=NonNegativeReals)
model.L_c = Var(domain=NonNegativeReals)
model.L_s = Var(domain=NonNegativeReals)
model.pw_w = Var(domain=NonNegativeReals)
model.pw_c = Var(domain=NonNegativeReals)

# Define constraints
model.land_con = Constraint(expr = model.L_w + model.L_c + model.L_s <= A)
model.prod_amount_wheat = Constraint(expr = model.L_w*y_w >= D_w - model.pw_w)
model.prod_amount_corn = Constraint(expr = model.L_c*y_c >= D_c - model.pw_c)
model.prod_amount_sugar_beets = Constraint(expr = model.L_s*y_s >= Q_s)
model.demand_req_wheat = Constraint(expr = y_w*model.L_w + model.pw_w >= D_w)
model.demand_req_corn = Constraint(expr = y_c*model.L_c + model.pw_c >= D_c)

# Define objective function (to minimize)
model.obj = Objective(expr = c_w*model.L_w + c_c*model.L_c + c_s*model.L_s + \
                      pw_w*model.pw_w + pw_c*model.pw_c - \
                      p_w*model.L_w*y_w - p_c*model.L_c*y_c - \
                      p_s*min(Q_s, model.L_s*y_s) - p_sq*max(0, model.L_s*y_s-Q_s), sense=minimize)

# Create a solver
solver = SolverFactory('glpk')

# Solve the model
solver.solve(model)

# print the results
print("Solution:")
print("L_w = ", model.L_w.value)
print("L_c = ", model.L_c.value)
print("L_s = ", model.L_s.value)
print("pw_w = ", model.pw_w.value)
print("pw_c = ", model.pw_c.value)

PyomoException: Cannot convert non-constant Pyomo expression (20*L_s  <  6000) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (if translation of model into code was incorrect or unexecutable)**

In [ ]:
from pyomo.environ import *
model = ConcreteModel()

# Define parameters (using sample data)
c_w = 150
c_c = 230
c_s = 260
A = 500
y_w = 2.5
y_c = 3
y_s = 20
p_w = 170
p_c = 150
p_s = 36
p_sq = 10
pw_w = p_w * 1.4
pw_c = p_c * 1.4
Q_s = 6000
D_w = 200
D_c = 240

# Define variables
model.L_w = Var(domain=NonNegativeReals)
model.L_c = Var(domain=NonNegativeReals)
model.L_s = Var(domain=NonNegativeReals)
model.pw_w = Var(domain=NonNegativeReals)
model.pw_c = Var(domain=NonNegativeReals)

# Define constraints
model.land_con = Constraint(expr = model.L_w + model.L_c + model.L_s <= A)
model.prod_amount_wheat = Constraint(expr = model.L_w*y_w >= D_w - model.pw_w)
model.prod_amount_corn = Constraint(expr = model.L_c*y_c >= D_c - model.pw_c)
model.prod_amount_sugar_beets = Constraint(expr = model.L_s*y_s >= Q_s)
model.demand_req_wheat = Constraint(expr = y_w*model.L_w + model.pw_w >= D_w)
model.demand_req_corn = Constraint(expr = y_c*model.L_c + model.pw_c >= D_c)

# Define objective function (to minimize)
model.obj = Objective(expr = c_w*model.L_w + c_c*model.L_c + c_s*model.L_s + \
                      pw_w*model.pw_w + pw_c*model.pw_c - \
                      p_w*model.L_w*y_w - p_c*model.L_c*y_c - \
                      p_s*min(Q_s, model.L_s*y_s) - p_sq*max(0, model.L_s*y_s-Q_s), sense=minimize)

# Create a solver
solver = SolverFactory('glpk')

# Solve the model
solver.solve(model)

# print the results
print("Solution:")
print("L_w = ", model.L_w.value)
print("L_c = ", model.L_c.value)
print("L_s = ", model.L_s.value)
print("pw_w = ", model.pw_w.value)
print("pw_c = ", model.pw_c.value)

PyomoException: Cannot convert non-constant Pyomo expression (20*L_s  <  6000) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.